In [ ]:
# the EarlyStopping and ModelCheckpoint callbacks
# EarlyStopping: allow you to interrupt training as soon as you start overfitting
# ModelCheckpoint: allow you to continually save the model during training, or save only the current best model
import keras

callbacks_list=[
    keras.callbacks.EarlyStopping(monitor='acc', patience=1,),
    keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True, )
]

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

In [ ]:
# the ReduceLROnPlateau callback
# reduce the learning rate when the validation loss has stopped improving

callbacks_list=[
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, )
]

model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val,y_val))

In [ ]:
# Write a custom callback
# self.model: the model instance from which the callback is being called
# self.validation_data: the value of what was passed to fit as validation data

import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model):
        # this method is called by the parent model before training, to inform the callback of what model will be calling it
        self.model=model
        layer_outputs=[layer.output for layer in model.layers]
        # this is a model instance that returns the activations of every layer
        self.activations_model=keras.models.Model(model.input, layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        # obtain first input sample of the validation data
        validation_sample=self.validation_data[0][0:1]
        activations=self.activations_model.predict(validation_sample)
        # save arrays to disk
        f=open('activations_at_epoch_'+str(epoch)+'.npz','w')
        np.savez(f,activations)
        f.close()

In [12]:
# Tensorboard Callback
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features=2000
max_len=500

(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)
x_train=sequence.pad_sequences(x_train,maxlen=max_len)
x_test=sequence.pad_sequences(x_test,maxlen=max_len)

model=keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

callbacks=[
    keras.callbacks.TensorBoard(log_dir='C:/Users/think/Desktop/keras/my_log_dir',
                                histogram_freq=1,embeddings_freq=1,)
]

history=model.fit(x_train,y_train,epochs=20,batch_size=128,validation_split=0.2,callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 98, 32)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_12 (Glo (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________
Trai